In [1]:
import pandas as pd

#### Load and Preview Excel Data ####

First, we start off by loading the two sheets for transactions CY 2009 and CY 2010 into Dataframes and observe what the data looks like

In [2]:
file_path = '../sample_data/transaction_data.xlsx'
transaction_data = pd.read_excel(file_path, sheet_name= None)
transaction_data.keys()

dict_keys(['Year 2009-2010', 'Year 2010-2011'])

In [3]:
transactions_2009 = transaction_data['Year 2009-2010']
transactions_2009.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom


In [4]:
transactions_2010 = transaction_data['Year 2010-2011']
transactions_2010.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


To better understand the size of the data, we want to take a look at number of rows in each excel sheet

In [5]:
print(f'2009-2010 data rows: {transactions_2009.shape[0]}')
print(f'2010-2011 data rows: {transactions_2010.shape[0]}')

2009-2010 data rows: 525461
2010-2011 data rows: 541910


#### Monthly Data Summary ####

Next, we want to take a look at the :
* Number of Unique Invoices per Month
* Number of Active Customers per month (Customers with at least one Invoice in a given month)

In [37]:
# Create a YearMonth column to aggregate data by Month and Year
transactions_2009['InvoiceDate'] = pd.to_datetime(transactions_2009['InvoiceDate'])
transactions_2009['YearMonth'] = transactions_2009['InvoiceDate'].dt.strftime('%Y-%m')

# Aggregate the data to get monthly unique invoices and active customers from 2009-2010
monthly_data_2009 = transactions_2009.groupby('YearMonth').agg(
    unique_invoices = ('Invoice', 'nunique'),
    active_customers = ('Customer ID', 'nunique')
)

monthly_data_2009.head(12)

,unique_invoices,active_customers
YearMonth,,
2009-12,2330,1045
2010-01,1633,786
2010-02,1969,807
2010-03,2367,1111
2010-04,1892,998
2010-05,2418,1062
2010-06,2216,1095
2010-07,2017,988
2010-08,1877,964


In [40]:
# Similarly, we can repeat the process for 2010-2011
transactions_2010['InvoiceDate'] = pd.to_datetime(transactions_2010['InvoiceDate'])
transactions_2010['YearMonth'] = transactions_2010['InvoiceDate'].dt.strftime('%Y-%m')

monthly_data_2010 = transactions_2010.groupby('YearMonth').agg(
    unique_invoices = ('Invoice', 'nunique'),
    active_customers = ('Customer ID', 'nunique')
)

monthly_data_2010.head(12)

,unique_invoices,active_customers
YearMonth,,
2010-12,2025,948
2011-01,1476,783
2011-02,1393,798
2011-03,1983,1020
2011-04,1744,899
2011-05,2162,1079
2011-06,2012,1051
2011-07,1927,993
2011-08,1737,980
